In [10]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Настройка для отображения
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Библиотеки успешно импортированы")


Библиотеки успешно импортированы


## 1. Анализ структуры файлов


In [11]:
# Определение путей к данным
data_path = Path("extracted_data")
csv_path = data_path / "csv" / "syn_csv"
json_path = data_path / "json" / "syn_json"
xml_path = data_path / "xml" / "syn_xml"

# Подсчет файлов в каждой папке
csv_files = list(csv_path.glob("*.csv"))
json_files = list(json_path.glob("*.json"))
xml_files = list(xml_path.glob("*.xml"))

print(f"CSV файлов: {len(csv_files)}")
print(f"JSON файлов: {len(json_files)}")
print(f"XML файлов: {len(xml_files)}")

# Анализ размеров файлов
def get_file_sizes(file_list):
    sizes = []
    for file in file_list:
        size_mb = file.stat().st_size / (1024 * 1024)
        sizes.append(size_mb)
    return sizes

csv_sizes = get_file_sizes(csv_files)
json_sizes = get_file_sizes(json_files)
xml_sizes = get_file_sizes(xml_files)

print(f"\nРазмеры CSV файлов (MB):")
print(f"  Минимум: {min(csv_sizes):.2f}")
print(f"  Максимум: {max(csv_sizes):.2f}")
print(f"  Средний: {np.mean(csv_sizes):.2f}")
print(f"  Общий размер: {sum(csv_sizes):.2f}")

print(f"\nРазмеры JSON файлов (MB):")
print(f"  Минимум: {min(json_sizes):.2f}")
print(f"  Максимум: {max(json_sizes):.2f}")
print(f"  Средний: {np.mean(json_sizes):.2f}")
print(f"  Общий размер: {sum(json_sizes):.2f}")

print(f"\nРазмеры XML файлов (MB):")
print(f"  Минимум: {min(xml_sizes):.2f}")
print(f"  Максимум: {max(xml_sizes):.2f}")
print(f"  Средний: {np.mean(xml_sizes):.2f}")
print(f"  Общий размер: {sum(xml_sizes):.2f}")


CSV файлов: 12
JSON файлов: 31
XML файлов: 15

Размеры CSV файлов (MB):
  Минимум: 332.02
  Максимум: 332.34
  Средний: 332.18
  Общий размер: 3986.16

Размеры JSON файлов (MB):
  Минимум: 239.46
  Максимум: 414.72
  Средний: 267.59
  Общий размер: 8295.36

Размеры XML файлов (MB):
  Минимум: 20.28
  Максимум: 958.83
  Средний: 786.73
  Общий размер: 11800.91


## 2. Анализ CSV данных


In [12]:
# Загрузка первого CSV файла для анализа структуры
sample_csv = csv_files[0]
print(f"Анализируем файл: {sample_csv.name}")

# Читаем только первые несколько строк для анализа структуры, устойчиво к проблемным строкам и разделителям
try:
    df_sample = pd.read_csv(
        sample_csv,
        nrows=1000,
        sep=None,              # авто-определение разделителя
        engine='python',       # более терпимый парсер
        on_bad_lines='skip',   # пропускать битые строки
        quoting=3              # QUOTE_NONE
    )
except Exception as e:
    print(f"Не удалось прочитать CSV стандартным способом: {e}\nПробую с запятой как разделителем...")
    df_sample = pd.read_csv(
        sample_csv,
        nrows=1000,
        sep=',',
        engine='python',
        on_bad_lines='skip'
    )

print(f"\nРазмер выборки: {df_sample.shape}")
print(f"\nКолонки:")
for i, col in enumerate(df_sample.columns):
    print(f"  {i+1}. {col}")

print(f"\nТипы данных:")
print(df_sample.dtypes)

print(f"\nПервые 5 строк:")
display(df_sample.head())

print(f"\nБазовая статистика:")
display(df_sample.describe(include='all'))


Анализируем файл: part-00000-37dced01-2ad2-48c8-a56d-54b4d8760599-c000.csv

Размер выборки: (950, 27)

Колонки:
  1. created
  2. order_status
  3. ticket_status
  4. ticket_price
  5. visitor_category
  6. event_id
  7. is_active
  8. valid_to
  9. count_visitor
  10. is_entrance
  11. is_entrance_mdate
  12. event_name
  13. event_kind_name
  14. spot_id
  15. spot_name
  16. museum_name
  17. start_datetime
  18. ticket_id
  19. update_timestamp
  20. client_name
  21. name
  22. surname
  23. client_phone
  24. museum_inn
  25. birthday_date
  26. order_number
  27. ticket_number

Типы данных:
created               object
order_status          object
ticket_status         object
ticket_price         float64
visitor_category      object
event_id               int64
is_active               bool
valid_to              object
count_visitor          int64
is_entrance             bool
is_entrance_mdate     object
event_name            object
event_kind_name       object
spot_id           

,created,order_status,ticket_status,ticket_price,visitor_category,event_id,is_active,valid_to,count_visitor,is_entrance,is_entrance_mdate,event_name,event_kind_name,spot_id,spot_name,museum_name,start_datetime,ticket_id,update_timestamp,client_name,name,surname,client_phone,museum_inn,birthday_date,order_number,ticket_number
0,2021-08-18T16:01:14.583+03:00,PAID,PAID,0.0,Обучающиеся по очной форме обучения в государс...,7561,True,2021-08-18,1,True,2021-08-18T19:14:45.427+03:00,Бальный танец,"консультация, методическое занятие, семинар, т...",274010,"Шверника ул. 13, корпус 2",Государственное бюджетное учреждение культуры ...,2021-08-18 17:00:00,1778482,2021-08-18T16:01:15.682+03:00,ШУКУРОВ РУСЛАН,КИРИЛЛ,ШУКУРОВ,7.985948e+10,3832203597,NaN,75343-483088,07a16922-969c-1033-a23f-f20b57dcf045
1,2021-09-16T18:45:54.401+03:00,PAID,PAID,250.0,Взрослые граждане РФ и СНГ,1765,True,2021-09-18,1,False,2021-09-16T18:45:54.977+03:00,Посещение Дворца (Большого дома),выставка,69,Усадьба Кусково,Государственное бюджетное учреждение культуры ...,2021-09-18 10:00:00,2100707,2021-09-16T18:46:19.713+03:00,КОБЕЛЬКОВА АЛЁНА,СЕРГЕЙ,КОБЕЛЬКОВ,7.926477e+10,415706141,NaN,04091-683930,2d6fcdd1-c16d-9501-bda8-ae329e98a5ca
2,2021-05-06T12:02:04.808+03:00,PAID,PAID,800.0,Взрослые граждане РФ и СНГ,1473,True,2021-05-06,1,True,2021-05-06T12:03:08.989+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-06 07:30:00,881269,2021-05-06T12:02:05.566+03:00,БОБРОВ ЮРИЙ,NaN,NaN,NaN,8719976264,NaN,87693-505050,06c34a5c-b6b7-1b92-b245-460c1a7ce3e9
3,2021-05-25T16:05:06.883+03:00,PAID,PAID,0.0,"Студенты, обучающиеся по очной форме обучения ...",1473,True,2021-05-25,1,True,2021-05-25T17:15:54.616+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-25 07:30:00,1147456,2021-05-25T16:05:07.733+03:00,КРЕБСА СОФИЯ,ИЛЬЯ,КРЕБС,7.911649e+10,1678121766,NaN,36421-684592,22f9764a-a036-2fb8-ac3a-2c98a58784b9
4,2021-05-07T14:49:50.473+03:00,PAID,PAID,0.0,Дети до 7 лет (не достигшие семилетнего возраста),1473,True,2021-05-07,1,False,2021-05-07T14:49:50.822+03:00,Посещение Московского зоопарка,выставка,276345,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-07 07:30:00,907812,2021-05-07T14:50:59.624+03:00,ПЕТРОСЯНА ГАЛИНА,ДМИТРИЙ,ПЕТРОСЯН,7.977650e+10,2599957179,NaN,72297-435006,441c1a65-1d3a-3e8a-85a4-be47e023f563



Базовая статистика:


,created,order_status,ticket_status,ticket_price,visitor_category,event_id,is_active,valid_to,count_visitor,is_entrance,is_entrance_mdate,event_name,event_kind_name,spot_id,spot_name,museum_name,start_datetime,ticket_id,update_timestamp,client_name,name,surname,client_phone,museum_inn,birthday_date,order_number,ticket_number
count,950,950,950,950.00000,950,950.000000,950,950,950.0,950,950,950,950,950.000000,950,950,923,9.500000e+02,950,950,684,684,7.270000e+02,9.500000e+02,165,950,950
unique,949,2,2,NaN,28,NaN,1,274,NaN,2,950,124,17,NaN,70,47,473,NaN,949,949,245,634,NaN,NaN,165,950,950
top,2021-09-30T16:15:53.622+03:00,PAID,PAID,NaN,Взрослые граждане РФ и СНГ,NaN,True,2021-08-22,NaN,True,2021-08-18T19:14:45.427+03:00,Посещение Московского зоопарка,выставка,NaN,"Грузинская Б. ул. 1, строение 1",Государственное автономное учреждение города М...,2021-05-07 07:30:00,NaN,2021-09-30T16:16:43.409+03:00,ЕФИМОВА МАРИЯ,АЛЕКСАНДР,ИВАНОВА,NaN,NaN,1989-10-28,75343-483088,07a16922-969c-1033-a23f-f20b57dcf045
freq,2,944,944,NaN,540,NaN,950,18,NaN,737,1,598,859,NaN,598,598,12,NaN,2,2,29,7,NaN,NaN,1,1,1
mean,NaN,NaN,NaN,321.80000,NaN,2004.008421,NaN,NaN,1.0,NaN,NaN,NaN,NaN,207230.994737,NaN,NaN,NaN,1.241817e+06,NaN,NaN,NaN,NaN,7.923116e+10,4.930066e+09,NaN,NaN,NaN
std,NaN,NaN,NaN,367.72708,NaN,1492.883859,NaN,NaN,0.0,NaN,NaN,NaN,NaN,119484.123682,NaN,NaN,NaN,6.601487e+05,NaN,NaN,NaN,NaN,7.041278e+08,2.864902e+09,NaN,NaN,NaN
min,NaN,NaN,NaN,0.00000,NaN,587.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,3.910200e+04,NaN,NaN,NaN,NaN,7.165523e+10,2.546349e+06,NaN,NaN,NaN
25%,NaN,NaN,NaN,0.00000,NaN,1473.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,271246.000000,NaN,NaN,NaN,6.561912e+05,NaN,NaN,NaN,NaN,7.910514e+10,2.491375e+09,NaN,NaN,NaN
50%,NaN,NaN,NaN,50.00000,NaN,1473.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,276345.000000,NaN,NaN,NaN,1.274764e+06,NaN,NaN,NaN,NaN,7.919925e+10,4.847933e+09,NaN,NaN,NaN
75%,NaN,NaN,NaN,800.00000,NaN,1963.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,276345.000000,NaN,NaN,NaN,1.816621e+06,NaN,NaN,NaN,NaN,7.958409e+10,7.314322e+09,NaN,NaN,NaN


## 3. Анализ JSON данных


In [13]:
# Анализ структуры JSON файла
sample_json = json_files[0]
print(f"Анализируем файл: {sample_json.name}")

# Читаем JSON файл
with open(sample_json, 'r', encoding='utf-8') as f:
    # Читаем только первые несколько строк для анализа структуры
    lines = []
    for i, line in enumerate(f):
        if i >= 10:  # Берем только первые 10 строк
            break
        lines.append(line.strip())

print(f"\nПервые 10 строк JSON файла:")
for i, line in enumerate(lines):
    print(f"{i+1}: {line[:200]}..." if len(line) > 200 else f"{i+1}: {line}")

# Пытаемся загрузить один JSON объект для анализа структуры
try:
    with open(sample_json, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        json_obj = json.loads(first_line)
        
    print(f"\nСтруктура JSON объекта:")
    if isinstance(json_obj, dict):
        print(f"Ключи: {list(json_obj.keys())}")
        for key, value in json_obj.items():
            print(f"  {key}: {type(value).__name__} - {str(value)[:100]}...")
    else:
        print(f"Тип данных: {type(json_obj).__name__}")
        print(f"Содержимое: {str(json_obj)[:200]}...")
        
except Exception as e:
    print(f"Ошибка при анализе JSON: {e}")


Анализируем файл: part1.json

Первые 10 строк JSON файла:
1: [
2: {
3: "date_exec": "2024-10-09",
4: "code_form_ind_entrep": "1",
5: "name_form_ind_entrep": "Индивидуальный предприниматель",
6: "inf_surname_ind_entrep_sex": "1",
7: "citizenship_kind": "1",
8: "inf_authority_reg_ind_entrep_name": "Управление Федеральной налоговой службы по Пензенской области",
9: "inf_authority_reg_ind_entrep_code": "5800",
10: "inf_reg_tax_ind_entrep": "Управление Федеральной налоговой службы по Пензенской области",
Ошибка при анализе JSON: Expecting value: line 1 column 2 (char 1)


## 4. Анализ XML данных


In [14]:
# Анализ структуры XML файла
sample_xml = xml_files[0]
print(f"Анализируем файл: {sample_xml.name}")

try:
    # Читаем XML файл
    tree = ET.parse(sample_xml)
    root = tree.getroot()
    
    print(f"\nКорневой элемент: {root.tag}")
    print(f"Атрибуты корневого элемента: {root.attrib}")
    
    # Анализируем структуру
    def analyze_xml_structure(element, level=0, max_level=3):
        if level > max_level:
            return
        
        indent = "  " * level
        print(f"{indent}{element.tag}")
        
        if element.attrib:
            print(f"{indent}  Атрибуты: {element.attrib}")
        
        if element.text and element.text.strip():
            text = element.text.strip()[:100]
            print(f"{indent}  Текст: {text}...")
        
        # Анализируем дочерние элементы (только первые 5)
        children = list(element)[:5]
        for child in children:
            analyze_xml_structure(child, level + 1, max_level)
        
        if len(list(element)) > 5:
            print(f"{indent}  ... и еще {len(list(element)) - 5} элементов")
    
    print(f"\nСтруктура XML (первые 3 уровня):")
    analyze_xml_structure(root)
    
except Exception as e:
    print(f"Ошибка при анализе XML: {e}")
    
    # Если не удается распарсить как XML, читаем как текст
    try:
        with open(sample_xml, 'r', encoding='utf-8') as f:
            content = f.read(1000)  # Читаем первые 1000 символов
            print(f"\nПервые 1000 символов файла:")
            print(content)
    except Exception as e2:
        print(f"Не удалось прочитать файл: {e2}")


Анализируем файл: part1.xml

Корневой элемент: root
Атрибуты корневого элемента: {}

Структура XML (первые 3 уровня):
root
  item
    actual_unverified_data
      purpose
    dated_info
      oti
    metadata
      last_change_record_number
        Текст: 50:21:0140218:1614-11...
      last_container_fixed_at
        Текст: 2019-01-09T00:00:00+03:00...
      status
        Текст: actual...
    object_common_data
      cad_number
        Текст: 50:21:0140218:1614...
      previously_posted
        Текст: true...
      quarter_cad_number
        Текст: 77:17:0140116...
      type
    object_formation
      method
    ... и еще 2 элементов
  item
    actual_unverified_data
      purpose
    dated_info
      oti
    metadata
      last_change_record_number
        Текст: 50:21:0140207:1336-24...
      last_container_fixed_at
        Текст: 2021-12-26T12:46:59+00:00...
      status
        Текст: actual...
    object_common_data
      cad_number
        Текст: 50:21:0140207:1336...
      pr

## 5. Сравнение данных в разных форматах


In [15]:
# Загружаем данные из всех форматов для сравнения
print("=== СРАВНЕНИЕ ДАННЫХ В РАЗНЫХ ФОРМАТАХ ===\n")

# Определяем пути к данным (если не определены ранее)
if 'csv_files' not in locals():
    data_path = Path("extracted_data")
    csv_path = data_path / "csv" / "syn_csv"
    json_path = data_path / "json" / "syn_json"
    xml_path = data_path / "xml" / "syn_xml"
    
    csv_files = list(csv_path.glob("*.csv"))
    json_files = list(json_path.glob("*.json"))
    xml_files = list(xml_path.glob("*.xml"))

# 1. Анализ CSV данных (музейные билеты)
print("1. CSV ДАННЫЕ (Музейные билеты):")
print("-" * 50)
csv_df = pd.read_csv(csv_files[0], nrows=1000, sep=',', engine='python', on_bad_lines='skip')
print(f"Количество записей: {len(csv_df)}")
print(f"Количество полей: {len(csv_df.columns)}")
print(f"Поля: {list(csv_df.columns)}")
print(f"Первая запись:")
print(csv_df.iloc[0].to_dict())
print()

# 2. Анализ JSON данных (предприниматели)
print("2. JSON ДАННЫЕ (Индивидуальные предприниматели):")
print("-" * 50)
# JSON файл содержит массив объектов, читаем по строкам
json_data = []
with open(json_files[0], 'r', encoding='utf-8') as f:
    content = f.read()
    # Убираем квадратные скобки и разбиваем по объектам
    content = content.strip()[1:-1]  # убираем [ и ]
    # Разбиваем по },{ но аккуратно
    objects = []
    current_obj = ""
    brace_count = 0
    for char in content:
        current_obj += char
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0:
                objects.append(current_obj.strip())
                current_obj = ""
    
    # Парсим первые несколько объектов
    for i, obj_str in enumerate(objects[:3]):
        try:
            obj = json.loads(obj_str)
            json_data.append(obj)
        except:
            continue

print(f"Количество записей (первые 3): {len(json_data)}")
if json_data:
    print(f"Количество полей: {len(json_data[0].keys())}")
    print(f"Поля: {list(json_data[0].keys())}")
    print(f"Первая запись:")
    for key, value in json_data[0].items():
        print(f"  {key}: {value}")
print()

# 3. Анализ XML данных (кадастровые данные)
print("3. XML ДАННЫЕ (Кадастровые данные):")
print("-" * 50)
tree = ET.parse(xml_files[0])
root = tree.getroot()
items = root.findall('item')
print(f"Количество записей: {len(items)}")
if items:
    first_item = items[0]
    print(f"Структура первой записи:")
    def print_xml_structure(element, level=0, max_level=2):
        if level > max_level:
            return
        indent = "  " * level
        print(f"{indent}{element.tag}")
        if element.text and element.text.strip():
            print(f"{indent}  Значение: {element.text.strip()[:50]}...")
        for child in element:
            print_xml_structure(child, level + 1, max_level)
    print_xml_structure(first_item)
print()

print("=== ВЫВОД: ДАННЫЕ РАЗНЫЕ! ===")
print("CSV: Музейные билеты и посетители")
print("JSON: Индивидуальные предприниматели") 
print("XML: Кадастровые данные недвижимости")


=== СРАВНЕНИЕ ДАННЫХ В РАЗНЫХ ФОРМАТАХ ===

1. CSV ДАННЫЕ (Музейные билеты):
--------------------------------------------------
Количество записей: 998
Количество полей: 1
Поля: ['created;order_status;ticket_status;ticket_price;visitor_category;event_id;is_active;valid_to;count_visitor;is_entrance;is_entrance_mdate;event_name;event_kind_name;spot_id;spot_name;museum_name;start_datetime;ticket_id;update_timestamp;client_name;name;surname;client_phone;museum_inn;birthday_date;order_number;ticket_number']
Первая запись:
{'created;order_status;ticket_status;ticket_price;visitor_category;event_id;is_active;valid_to;count_visitor;is_entrance;is_entrance_mdate;event_name;event_kind_name;spot_id;spot_name;museum_name;start_datetime;ticket_id;update_timestamp;client_name;name;surname;client_phone;museum_inn;birthday_date;order_number;ticket_number': ' корпус 2;Государственное бюджетное учреждение культуры города Москвы «Центр культуры и досуга «Академический»;2021-08-18 17:00:00;1778482;2021-08

## 6. Анализ качества данных и поиск ошибок


In [16]:
# Анализ качества CSV данных (музейные билеты)
print("=== АНАЛИЗ КАЧЕСТВА CSV ДАННЫХ ===")
print()

# Определяем пути к данным (если не определены ранее)
if 'csv_files' not in locals():
    data_path = Path("extracted_data")
    csv_path = data_path / "csv" / "syn_csv"
    json_path = data_path / "json" / "syn_json"
    xml_path = data_path / "xml" / "syn_xml"
    
    csv_files = list(csv_path.glob("*.csv"))
    json_files = list(json_path.glob("*.json"))
    xml_files = list(xml_path.glob("*.xml"))

# Загружаем полный датасет
csv_df_full = pd.read_csv(csv_files[0], sep=',', engine='python', on_bad_lines='skip')
print(f"Общее количество записей: {len(csv_df_full)}")
print(f"Общее количество полей: {len(csv_df_full.columns)}")
print()

# 1. Проверка на пропущенные значения
print("1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:")
print("-" * 30)
missing_data = csv_df_full.isnull().sum()
missing_percent = (missing_data / len(csv_df_full)) * 100
missing_df = pd.DataFrame({
    'Поле': missing_data.index,
    'Пропущено': missing_data.values,
    'Процент': missing_percent.values
}).sort_values('Пропущено', ascending=False)

print(missing_df[missing_df['Пропущено'] > 0])
print()

# 2. Проверка дубликатов
print("2. ДУБЛИКАТЫ:")
print("-" * 15)
duplicates = csv_df_full.duplicated().sum()
print(f"Полные дубликаты: {duplicates}")
print(f"Процент дубликатов: {(duplicates/len(csv_df_full)*100):.2f}%")
print()

# 3. Анализ уникальных значений в ключевых полях
print("3. АНАЛИЗ УНИКАЛЬНЫХ ЗНАЧЕНИЙ:")
print("-" * 35)
key_fields = ['ticket_id', 'order_number', 'ticket_number', 'client_phone', 'museum_inn']
for field in key_fields:
    if field in csv_df_full.columns:
        unique_count = csv_df_full[field].nunique()
        total_count = len(csv_df_full)
        print(f"{field}: {unique_count} уникальных из {total_count} ({unique_count/total_count*100:.1f}%)")
print()

# 4. Проверка логических ошибок
print("4. ЛОГИЧЕСКИЕ ПРОВЕРКИ:")
print("-" * 25)

# Проверка цен билетов
if 'ticket_price' in csv_df_full.columns:
    price_stats = csv_df_full['ticket_price'].describe()
    print(f"Цены билетов: мин={price_stats['min']:.2f}, макс={price_stats['max']:.2f}, среднее={price_stats['mean']:.2f}")
    
    # Проверка на отрицательные цены
    negative_prices = (csv_df_full['ticket_price'] < 0).sum()
    print(f"Отрицательные цены: {negative_prices}")
    
    # Проверка на нулевые цены
    zero_prices = (csv_df_full['ticket_price'] == 0).sum()
    print(f"Нулевые цены: {zero_prices}")

# Проверка дат
date_fields = ['created', 'valid_to', 'start_datetime', 'birthday_date', 'is_entrance_mdate']
for field in date_fields:
    if field in csv_df_full.columns:
        try:
            # Пытаемся преобразовать в datetime
            pd.to_datetime(csv_df_full[field], errors='coerce')
            invalid_dates = pd.to_datetime(csv_df_full[field], errors='coerce').isnull().sum()
            print(f"{field}: {invalid_dates} невалидных дат")
        except:
            print(f"{field}: не удалось проверить даты")

print()

# 5. Проверка телефонных номеров
print("5. ПРОВЕРКА ТЕЛЕФОННЫХ НОМЕРОВ:")
print("-" * 35)
if 'client_phone' in csv_df_full.columns:
    phone_stats = csv_df_full['client_phone'].describe()
    print(f"Статистика телефонов: {phone_stats}")
    
    # Проверка на российские номера (должны начинаться с 7 или 8)
    phone_str = csv_df_full['client_phone'].astype(str)
    russian_phones = phone_str.str.match(r'^[78]\d{10}$').sum()
    print(f"Российские номера (7XXXXXXXXXX или 8XXXXXXXXXX): {russian_phones}")
    
    # Проверка на короткие номера
    short_phones = phone_str.str.len() < 10
    print(f"Короткие номера (< 10 цифр): {short_phones.sum()}")
    
    # Проверка на длинные номера
    long_phones = phone_str.str.len() > 15
    print(f"Длинные номера (> 15 цифр): {long_phones.sum()}")

print()

# 6. Проверка ИНН музеев
print("6. ПРОВЕРКА ИНН МУЗЕЕВ:")
print("-" * 25)
if 'museum_inn' in csv_df_full.columns:
    inn_stats = csv_df_full['museum_inn'].describe()
    print(f"Статистика ИНН: {inn_stats}")
    
    # Проверка длины ИНН (должны быть 10 или 12 цифр)
    inn_str = csv_df_full['museum_inn'].astype(str)
    valid_inn_length = inn_str.str.len().isin([10, 12]).sum()
    print(f"ИНН правильной длины (10 или 12 цифр): {valid_inn_length}")
    
    # Проверка на числовые ИНН
    numeric_inn = inn_str.str.isdigit().sum()
    print(f"Числовые ИНН: {numeric_inn}")

print()


=== АНАЛИЗ КАЧЕСТВА CSV ДАННЫХ ===

Общее количество записей: 478344
Общее количество полей: 1

1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:
------------------------------
                                                Поле  Пропущено    Процент
0  created;order_status;ticket_status;ticket_pric...     477909  99.909061

2. ДУБЛИКАТЫ:
---------------
Полные дубликаты: 477908
Процент дубликатов: 99.91%

3. АНАЛИЗ УНИКАЛЬНЫХ ЗНАЧЕНИЙ:
-----------------------------------

4. ЛОГИЧЕСКИЕ ПРОВЕРКИ:
-------------------------

5. ПРОВЕРКА ТЕЛЕФОННЫХ НОМЕРОВ:
-----------------------------------

6. ПРОВЕРКА ИНН МУЗЕЕВ:
-------------------------



In [17]:
# Анализ качества JSON данных (предприниматели)
print("=== АНАЛИЗ КАЧЕСТВА JSON ДАННЫХ ===")
print()

# Определяем пути к данным (если не определены ранее)
if 'json_files' not in locals():
    data_path = Path("extracted_data")
    csv_path = data_path / "csv" / "syn_csv"
    json_path = data_path / "json" / "syn_json"
    xml_path = data_path / "xml" / "syn_xml"
    
    csv_files = list(csv_path.glob("*.csv"))
    json_files = list(json_path.glob("*.json"))
    xml_files = list(xml_path.glob("*.xml"))

# Загружаем несколько JSON файлов для анализа
json_data_all = []
for json_file in json_files[:3]:  # Анализируем первые 3 файла
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            content = f.read()
            # Убираем квадратные скобки и разбиваем по объектам
            content = content.strip()[1:-1]
            objects = []
            current_obj = ""
            brace_count = 0
            for char in content:
                current_obj += char
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        objects.append(current_obj.strip())
                        current_obj = ""
            
            # Парсим объекты
            for obj_str in objects:
                try:
                    obj = json.loads(obj_str)
                    json_data_all.append(obj)
                except:
                    continue
    except Exception as e:
        print(f"Ошибка при чтении {json_file}: {e}")

print(f"Загружено записей из JSON: {len(json_data_all)}")
if json_data_all:
    # Преобразуем в DataFrame для анализа
    json_df = pd.DataFrame(json_data_all)
    print(f"Количество полей: {len(json_df.columns)}")
    print(f"Поля: {list(json_df.columns)}")
    print()
    
    # Проверка на пропущенные значения
    print("1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:")
    print("-" * 30)
    missing_data = json_df.isnull().sum()
    missing_percent = (missing_data / len(json_df)) * 100
    missing_df = pd.DataFrame({
        'Поле': missing_data.index,
        'Пропущено': missing_data.values,
        'Процент': missing_percent.values
    }).sort_values('Пропущено', ascending=False)
    
    print(missing_df[missing_df['Пропущено'] > 0])
    print()
    
    # Анализ ключевых полей
    print("2. АНАЛИЗ КЛЮЧЕВЫХ ПОЛЕЙ:")
    print("-" * 30)
    key_fields = ['inf_surname_ind_entrep_sex', 'citizenship_kind', 'code_form_ind_entrep']
    for field in key_fields:
        if field in json_df.columns:
            unique_count = json_df[field].nunique()
            total_count = len(json_df)
            print(f"{field}: {unique_count} уникальных из {total_count}")
            print(f"  Значения: {json_df[field].value_counts().head().to_dict()}")
    print()

# Анализ качества XML данных (кадастровые данные)
print("=== АНАЛИЗ КАЧЕСТВА XML ДАННЫХ ===")
print()

# Анализируем несколько XML файлов
xml_data_all = []
for xml_file in xml_files[:3]:  # Анализируем первые 3 файла
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        items = root.findall('item')
        xml_data_all.extend(items)
    except Exception as e:
        print(f"Ошибка при чтении {xml_file}: {e}")

print(f"Загружено записей из XML: {len(xml_data_all)}")
if xml_data_all:
    # Анализируем структуру и качество данных
    print("1. СТРУКТУРА ДАННЫХ:")
    print("-" * 20)
    first_item = xml_data_all[0]
    
    # Собираем все уникальные теги
    all_tags = set()
    for item in xml_data_all[:100]:  # Анализируем первые 100 записей
        for elem in item.iter():
            all_tags.add(elem.tag)
    
    print(f"Уникальных тегов: {len(all_tags)}")
    print(f"Основные теги: {sorted(list(all_tags))[:10]}")
    print()
    
    # Проверка на пустые значения
    print("2. ПРОВЕРКА ПУСТЫХ ЗНАЧЕНИЙ:")
    print("-" * 30)
    empty_values = 0
    total_values = 0
    for item in xml_data_all[:100]:
        for elem in item.iter():
            total_values += 1
            if elem.text is None or elem.text.strip() == '':
                empty_values += 1
    
    print(f"Пустых значений: {empty_values} из {total_values} ({empty_values/total_values*100:.1f}%)")
    print()
    
    # Анализ кадастровых номеров
    print("3. АНАЛИЗ КАДАСТРОВЫХ НОМЕРОВ:")
    print("-" * 35)
    cad_numbers = []
    for item in xml_data_all[:100]:
        cad_number_elem = item.find('.//cad_number')
        if cad_number_elem is not None and cad_number_elem.text:
            cad_numbers.append(cad_number_elem.text.strip())
    
    print(f"Найдено кадастровых номеров: {len(cad_numbers)}")
    if cad_numbers:
        # Проверяем формат кадастровых номеров (должен быть XX:XX:XXXXXXX:XXX)
        valid_format = 0
        for cad_num in cad_numbers:
            if ':' in cad_num and len(cad_num.split(':')) == 4:
                valid_format += 1
        
        print(f"Кадастровых номеров правильного формата: {valid_format} из {len(cad_numbers)}")
        print(f"Примеры: {cad_numbers[:3]}")

print()


=== АНАЛИЗ КАЧЕСТВА JSON ДАННЫХ ===

Загружено записей из JSON: 3
Количество полей: 24
Поля: ['date_exec', 'code_form_ind_entrep', 'name_form_ind_entrep', 'inf_surname_ind_entrep_sex', 'citizenship_kind', 'inf_authority_reg_ind_entrep_name', 'inf_authority_reg_ind_entrep_code', 'inf_reg_tax_ind_entrep', 'inf_okved_code', 'inf_okved_name', 'process_dttm', 'error_code', 'inf_surname_ind_entrep_firstname', 'inf_surname_ind_entrep_surname', 'inf_surname_ind_entrep_midname', 'dob', 'date_ogrnip', 'id_card', 'innfl', 'ogrnip', 'inf_okved_opt', 'insured_pf', 'email_ind_entrep', 'insuref_fss']

1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:
------------------------------
           Поле  Пропущено    Процент
23  insuref_fss          2  66.666667

2. АНАЛИЗ КЛЮЧЕВЫХ ПОЛЕЙ:
------------------------------
inf_surname_ind_entrep_sex: 2 уникальных из 3
  Значения: {'1': 2, '2': 1}
citizenship_kind: 1 уникальных из 3
  Значения: {'1': 3}
code_form_ind_entrep: 1 уникальных из 3
  Значения: {'1': 3}

=== АНАЛИЗ КАЧЕСТВА XM

## 7. Глубокий анализ контекста и содержания данных


In [18]:
# Глубокий анализ контекста данных
print("=== ГЛУБОКИЙ АНАЛИЗ КОНТЕКСТА ДАННЫХ ===")
print()

# Определяем пути к данным и загружаем их (если не определены ранее)
if 'csv_df_full' not in locals():
    data_path = Path("extracted_data")
    csv_path = data_path / "csv" / "syn_csv"
    json_path = data_path / "json" / "syn_json"
    xml_path = data_path / "xml" / "syn_xml"
    
    csv_files = list(csv_path.glob("*.csv"))
    json_files = list(json_path.glob("*.json"))
    xml_files = list(xml_path.glob("*.xml"))
    
    # Загружаем CSV данные
    csv_df_full = pd.read_csv(csv_files[0], sep=',', engine='python', on_bad_lines='skip')
    
    # Загружаем JSON данные
    json_data_all = []
    for json_file in json_files[:3]:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                content = f.read()
                content = content.strip()[1:-1]
                objects = []
                current_obj = ""
                brace_count = 0
                for char in content:
                    current_obj += char
                    if char == '{':
                        brace_count += 1
                    elif char == '}':
                        brace_count -= 1
                        if brace_count == 0:
                            objects.append(current_obj.strip())
                            current_obj = ""
                
                for obj_str in objects:
                    try:
                        obj = json.loads(obj_str)
                        json_data_all.append(obj)
                    except:
                        continue
        except Exception as e:
            print(f"Ошибка при чтении {json_file}: {e}")
    
    # Загружаем XML данные
    xml_data_all = []
    for xml_file in xml_files[:3]:
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            items = root.findall('item')
            xml_data_all.extend(items)
        except Exception as e:
            print(f"Ошибка при чтении {xml_file}: {e}")

# 1. Анализ CSV данных (музейные билеты)
print("1. CSV ДАННЫЕ - МУЗЕЙНАЯ СИСТЕМА:")
print("=" * 50)

# Анализ музеев
if 'museum_name' in csv_df_full.columns:
    museums = csv_df_full['museum_name'].value_counts()
    print(f"Количество уникальных музеев: {len(museums)}")
    print(f"Топ-5 музеев по количеству билетов:")
    for i, (museum, count) in enumerate(museums.head().items()):
        print(f"  {i+1}. {museum}: {count} билетов")
    print()

# Анализ событий
if 'event_name' in csv_df_full.columns:
    events = csv_df_full['event_name'].value_counts()
    print(f"Количество уникальных событий: {len(events)}")
    print(f"Топ-5 событий:")
    for i, (event, count) in enumerate(events.head().items()):
        print(f"  {i+1}. {event}: {count} билетов")
    print()

# Анализ категорий посетителей
if 'visitor_category' in csv_df_full.columns:
    categories = csv_df_full['visitor_category'].value_counts()
    print(f"Категории посетителей:")
    for category, count in categories.items():
        print(f"  {category}: {count} ({count/len(csv_df_full)*100:.1f}%)")
    print()

# Анализ статусов заказов и билетов
if 'order_status' in csv_df_full.columns:
    order_statuses = csv_df_full['order_status'].value_counts()
    print(f"Статусы заказов:")
    for status, count in order_statuses.items():
        print(f"  {status}: {count} ({count/len(csv_df_full)*100:.1f}%)")
    print()

if 'ticket_status' in csv_df_full.columns:
    ticket_statuses = csv_df_full['ticket_status'].value_counts()
    print(f"Статусы билетов:")
    for status, count in ticket_statuses.items():
        print(f"  {status}: {count} ({count/len(csv_df_full)*100:.1f}%)")
    print()

# Анализ временных периодов
if 'created' in csv_df_full.columns:
    try:
        csv_df_full['created_dt'] = pd.to_datetime(csv_df_full['created'], errors='coerce')
        date_range = csv_df_full['created_dt'].agg(['min', 'max'])
        print(f"Период данных: с {date_range['min'].strftime('%Y-%m-%d')} по {date_range['max'].strftime('%Y-%m-%d')}")
        
        # Анализ по месяцам
        monthly_stats = csv_df_full.groupby(csv_df_full['created_dt'].dt.to_period('M')).size()
        print(f"Активность по месяцам:")
        for period, count in monthly_stats.head().items():
            print(f"  {period}: {count} билетов")
    except:
        print("Не удалось проанализировать временные периоды")
    print()

# 2. Анализ JSON данных (предприниматели)
print("2. JSON ДАННЫЕ - РЕЕСТР ИНДИВИДУАЛЬНЫХ ПРЕДПРИНИМАТЕЛЕЙ:")
print("=" * 70)

if json_data_all:
    json_df = pd.DataFrame(json_data_all)
    
    # Анализ форм предпринимательства
    if 'name_form_ind_entrep' in json_df.columns:
        forms = json_df['name_form_ind_entrep'].value_counts()
        print(f"Формы предпринимательства:")
        for form, count in forms.items():
            print(f"  {form}: {count} ({count/len(json_df)*100:.1f}%)")
        print()
    
    # Анализ полов предпринимателей
    if 'inf_surname_ind_entrep_sex' in json_df.columns:
        sexes = json_df['inf_surname_ind_entrep_sex'].value_counts()
        print(f"Пол предпринимателей:")
        sex_mapping = {'1': 'Мужской', '2': 'Женский', '0': 'Не указан'}
        for sex_code, count in sexes.items():
            sex_name = sex_mapping.get(str(sex_code), f'Код {sex_code}')
            print(f"  {sex_name}: {count} ({count/len(json_df)*100:.1f}%)")
        print()
    
    # Анализ гражданства
    if 'citizenship_kind' in json_df.columns:
        citizenships = json_df['citizenship_kind'].value_counts()
        print(f"Гражданство:")
        citizenship_mapping = {'1': 'Российское', '2': 'Иностранное', '0': 'Не указано'}
        for citizenship_code, count in citizenships.items():
            citizenship_name = citizenship_mapping.get(str(citizenship_code), f'Код {citizenship_code}')
            print(f"  {citizenship_name}: {count} ({count/len(json_df)*100:.1f}%)")
        print()
    
    # Анализ налоговых органов
    if 'inf_authority_reg_ind_entrep_name' in json_df.columns:
        authorities = json_df['inf_authority_reg_ind_entrep_name'].value_counts()
        print(f"Топ-5 налоговых органов по количеству регистраций:")
        for i, (authority, count) in enumerate(authorities.head().items()):
            print(f"  {i+1}. {authority}: {count}")
        print()

# 3. Анализ XML данных (кадастровые данные)
print("3. XML ДАННЫЕ - КАДАСТРОВЫЕ ДАННЫЕ НЕДВИЖИМОСТИ:")
print("=" * 55)

if xml_data_all:
    # Анализ типов объектов
    object_types = []
    for item in xml_data_all[:100]:
        type_elem = item.find('.//type')
        if type_elem is not None and type_elem.text:
            object_types.append(type_elem.text.strip())
    
    if object_types:
        type_counts = pd.Series(object_types).value_counts()
        print(f"Типы недвижимых объектов:")
        for obj_type, count in type_counts.items():
            print(f"  {obj_type}: {count} ({count/len(object_types)*100:.1f}%)")
        print()
    
    # Анализ статусов объектов
    object_statuses = []
    for item in xml_data_all[:100]:
        status_elem = item.find('.//status')
        if status_elem is not None and status_elem.text:
            object_statuses.append(status_elem.text.strip())
    
    if object_statuses:
        status_counts = pd.Series(object_statuses).value_counts()
        print(f"Статусы объектов:")
        for status, count in status_counts.items():
            print(f"  {status}: {count} ({count/len(object_statuses)*100:.1f}%)")
        print()
    
    # Анализ регионов по кадастровым номерам
    regions = []
    for item in xml_data_all[:100]:
        cad_elem = item.find('.//cad_number')
        if cad_elem is not None and cad_elem.text:
            cad_num = cad_elem.text.strip()
            if ':' in cad_num:
                region_code = cad_num.split(':')[0]
                regions.append(region_code)
    
    if regions:
        region_counts = pd.Series(regions).value_counts()
        print(f"Топ-5 регионов по количеству объектов:")
        for i, (region, count) in enumerate(region_counts.head().items()):
            print(f"  {i+1}. Регион {region}: {count} объектов")
        print()

print("=== ОБЩИЕ ВЫВОДЫ ===")
print("1. CSV: Данные музейной системы - билеты, посетители, события")
print("2. JSON: Реестр индивидуальных предпринимателей - регистрационные данные")
print("3. XML: Кадастровые данные недвижимости - объекты, их характеристики")
print()
print("Все три набора данных РАЗНЫЕ и представляют разные домены!")


=== ГЛУБОКИЙ АНАЛИЗ КОНТЕКСТА ДАННЫХ ===

1. CSV ДАННЫЕ - МУЗЕЙНАЯ СИСТЕМА:
2. JSON ДАННЫЕ - РЕЕСТР ИНДИВИДУАЛЬНЫХ ПРЕДПРИНИМАТЕЛЕЙ:
Формы предпринимательства:
  Индивидуальный предприниматель: 3 (100.0%)

Пол предпринимателей:
  Мужской: 2 (66.7%)
  Женский: 1 (33.3%)

Гражданство:
  Российское: 3 (100.0%)

Топ-5 налоговых органов по количеству регистраций:
  1. Управление Федеральной налоговой службы по Пензенской области: 1
  2. Межрайонная инспекция Федеральной налоговой службы № 10 по Оренбургской области: 1
  3. Межрайонная инспекция Федеральной налоговой службы № 11 по Ханты-Мансийскому автономному округу - Югре: 1

3. XML ДАННЫЕ - КАДАСТРОВЫЕ ДАННЫЕ НЕДВИЖИМОСТИ:
Типы недвижимых объектов:
  : 100 (100.0%)

Статусы объектов:
  actual: 95 (95.0%)
  archived: 5 (5.0%)

Топ-5 регионов по количеству объектов:
  1. Регион 77: 78 объектов
  2. Регион 50: 22 объектов

=== ОБЩИЕ ВЫВОДЫ ===
1. CSV: Данные музейной системы - билеты, посетители, события
2. JSON: Реестр индивидуальных предп

In [19]:
# Дополнительная ячейка для проверки импортов и переменных
print("Проверка импортов и переменных...")

# Проверяем, что все библиотеки импортированы
try:
    print(f"pandas: {pd.__version__}")
    print(f"numpy: {np.__version__}")
    print(f"json: доступен")
    print(f"xml.etree.ElementTree: доступен")
    print(f"pathlib.Path: доступен")
    print(f"matplotlib: доступен")
    print(f"seaborn: доступен")
except NameError as e:
    print(f"Ошибка импорта: {e}")
    print("Запустите ячейку с импортами сначала!")

# Проверяем наличие переменных
print(f"\nПеременные:")
print(f"csv_files определены: {'csv_files' in locals()}")
print(f"json_files определены: {'json_files' in locals()}")
print(f"xml_files определены: {'xml_files' in locals()}")
print(f"csv_df_full определены: {'csv_df_full' in locals()}")
print(f"json_data_all определены: {'json_data_all' in locals()}")
print(f"xml_data_all определены: {'xml_data_all' in locals()}")


Проверка импортов и переменных...
pandas: 2.3.2
numpy: 2.3.3
json: доступен
xml.etree.ElementTree: доступен
pathlib.Path: доступен
matplotlib: доступен
seaborn: доступен

Переменные:
csv_files определены: True
json_files определены: True
xml_files определены: True
csv_df_full определены: True
json_data_all определены: True
xml_data_all определены: True


## 8. Упрощенный анализ (независимый от порядка выполнения)


In [20]:
# Упрощенный анализ, который работает независимо от порядка выполнения ячеек
print("=== УПРОЩЕННЫЙ АНАЛИЗ ДАННЫХ ===")
print()

# Импорты (на всякий случай)
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET
from pathlib import Path

# Определение путей
data_path = Path("extracted_data")
csv_path = data_path / "csv" / "syn_csv"
json_path = data_path / "json" / "syn_json"
xml_path = data_path / "xml" / "syn_xml"

csv_files = list(csv_path.glob("*.csv"))
json_files = list(json_path.glob("*.json"))
xml_files = list(xml_path.glob("*.xml"))

print(f"Найдено файлов:")
print(f"  CSV: {len(csv_files)}")
print(f"  JSON: {len(json_files)}")
print(f"  XML: {len(xml_files)}")
print()

# 1. Анализ CSV
print("1. CSV АНАЛИЗ:")
print("-" * 20)
if csv_files:
    try:
        df = pd.read_csv(csv_files[0], nrows=100, sep=',', engine='python', on_bad_lines='skip')
        print(f"Записей в выборке: {len(df)}")
        print(f"Полей: {len(df.columns)}")
        print(f"Поля: {list(df.columns)[:10]}...")  # Показываем первые 10 полей
        
        # Проверяем на пропуски
        missing = df.isnull().sum().sum()
        print(f"Пропущенных значений: {missing}")
        
        # Проверяем дубликаты
        duplicates = df.duplicated().sum()
        print(f"Дубликатов: {duplicates}")
        
    except Exception as e:
        print(f"Ошибка при чтении CSV: {e}")
print()

# 2. Анализ JSON
print("2. JSON АНАЛИЗ:")
print("-" * 20)
if json_files:
    try:
        with open(json_files[0], 'r', encoding='utf-8') as f:
            content = f.read(1000)  # Читаем первые 1000 символов
            print(f"Первые 200 символов: {content[:200]}...")
            
        # Пытаемся загрузить один объект
        with open(json_files[0], 'r', encoding='utf-8') as f:
            first_line = f.readline().strip()
            if first_line.startswith('['):
                # Это массив, читаем следующую строку
                second_line = f.readline().strip()
                if second_line.startswith('{'):
                    obj = json.loads(second_line)
                    print(f"Ключи в JSON объекте: {list(obj.keys())[:10]}...")
                    
    except Exception as e:
        print(f"Ошибка при чтении JSON: {e}")
print()

# 3. Анализ XML
print("3. XML АНАЛИЗ:")
print("-" * 20)
if xml_files:
    try:
        tree = ET.parse(xml_files[0])
        root = tree.getroot()
        print(f"Корневой элемент: {root.tag}")
        
        items = root.findall('item')
        print(f"Количество элементов item: {len(items)}")
        
        if items:
            first_item = items[0]
            print(f"Теги в первом элементе:")
            for child in first_item[:5]:  # Показываем первые 5 дочерних элементов
                print(f"  {child.tag}")
                
    except Exception as e:
        print(f"Ошибка при чтении XML: {e}")
print()

print("=== ВЫВОДЫ ===")
print("1. CSV: Музейные билеты и посетители")
print("2. JSON: Реестр индивидуальных предпринимателей") 
print("3. XML: Кадастровые данные недвижимости")
print()
print("Данные в разных форматах РАЗНЫЕ по содержанию!")


=== УПРОЩЕННЫЙ АНАЛИЗ ДАННЫХ ===

Найдено файлов:
  CSV: 12
  JSON: 31
  XML: 15

1. CSV АНАЛИЗ:
--------------------
Записей в выборке: 100
Полей: 1
Поля: ['created;order_status;ticket_status;ticket_price;visitor_category;event_id;is_active;valid_to;count_visitor;is_entrance;is_entrance_mdate;event_name;event_kind_name;spot_id;spot_name;museum_name;start_datetime;ticket_id;update_timestamp;client_name;name;surname;client_phone;museum_inn;birthday_date;order_number;ticket_number']...
Пропущенных значений: 99
Дубликатов: 98

2. JSON АНАЛИЗ:
--------------------
Первые 200 символов: [
  {
    "date_exec": "2024-10-09",
    "code_form_ind_entrep": "1",
    "name_form_ind_entrep": "Индивидуальный предприниматель",
    "inf_surname_ind_entrep_sex": "1",
    "citizenship_kind": "1",
...
Ошибка при чтении JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

3. XML АНАЛИЗ:
--------------------
Корневой элемент: root
Количество элементов item: 189441
Теги в первом 